In [ ]:
using Pkg
Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

In [ ]:
cd("\\\\inst3\\data\\Benutzer\\bergermann\\Desktop\\Julia Files\\JuXIMC.jl")

using Dragoon
using Dates
using Plots
using JLD2
using LaTeXStrings


include(joinpath(pwd(),"src/vna_control.jl"));
include(joinpath(pwd(),"src/JuXIMC.jl"));
include(joinpath(pwd(),"examplesdom/dragoonstuff.jl"));

include("stages2discs_c.jl");

In [ ]:
vna = connectVNA()
setupFromFile(vna,"src/vna_22G_50M.txt")

freqs = collect(Float64,getFreqAsBinBlockTransfer(vna))

function objRefVNA(booster::Booster,freqs::Vector{Float64},(vna,ref0)::Tuple{TCPSocket,Vector{ComplexF64}})
    # sleep(1)

    ref = getTrace(vna)

    return sum(abs.(ref-ref0))
end

function objRefRef(booster::Booster,freqs::Vector{Float64},(ref,ref0)::Tuple{Vector{ComplexF64},Vector{ComplexF64}})
    return sum(abs.(ref-ref0))
end

ObjRefVNA(vna,ref0) = Callback(objRefVNA,(vna,ref0))
ObjRefRef(ref,ref0) = Callback(objRefRef,(ref,ref0))

In [ ]:
infoXIMC();

D = requestDevices("Motor 7","Monica")

In [ ]:
devices = Devices(D,stagecals,stagecols,stagezeros,stageborders,stagehomes);
b = PhysicalBooster(devices, τ=1e-3,ϵ=9.4,maxlength=0.2);

homeHome(b)

In [ ]:
d0 = findpeak(22.025e9,2; eps=9.4,thickness=1e-3,dev=0.3,gran=10000)
d0 *= 2.7799

In [ ]:
# sleep(5)

move(b,[d0,2d0+1e-3]; additive=false)

In [ ]:
b.pos

In [ ]:
move(b,[d0,2d0+1e-3]; additive=false)

ref0 = getTrace(vna)
objF = ObjRefVNA(vna,ref0)
objFR(ref) = ObjRefRef(ref,ref0)

steps = 10

hist = initHist(b,(2*steps+1)^2+1,freqs,objFR(ref0));
updateHist!(b,hist,freqs,objFR(ref0))
R = zeros(ComplexF64,2*steps+1,2*steps+1,length(ref0))

p0 = copy(b.pos)

@time for i in -steps:steps, j in -steps:steps
    j == -steps && println("i: ",i)

    move(b,p0+[i*0.0005,j*0.0005]; additive=false)
    
    ref = getTrace(vna)
    updateHist!(b,hist,freqs,objFR(ref))
    R[i+steps+1,j+steps+1,:] = ref
end

@save "2_discs_c_scan_peak.jld2" ref0 hist R freqs

In [ ]:
@load "2_discs_c_scan_peak.jld2"

R_ = reshape((x->x.objvalue).(hist[1:end-1]),(21,21))

scan = contourf(-5:0.5:5,-5:0.5:5,log10.(R_); color=:turbo,levels=20,lw=0,aspect_ratio=:equal,clim=(0,1.8),colorbar_title=L"\log_{10} f_R")
plot!([-6,-6,2.5,2.5,-6],[-2.5,4,4,-2.5,-2.5]; c=:black,linestyle=:dash,legend=false)
xlims!((-5,5))
ylims!((-5,5))
xlabel!(L"\Delta x_1"*" [mm]")
ylabel!(L"\Delta x_2"*" [mm]")
savefig("2disc_c_scan_p.pdf")

In [ ]:
[d0,2d0+1e-3]

In [ ]:
b.pos

In [ ]:
move(b,[d0,2d0+1e-3]; additive=false)

ref0 = getTrace(vna)
objF = ObjRefVNA(vna,ref0)
objFR(ref) = ObjRefRef(ref,ref0)

hist = initHist(b,1001,freqs,objFR(ref0));
updateHist!(b,hist,freqs,objFR(ref0))

move(b,[0.002,0.002]; additive=true)

trace = linesearch(b,hist,freqs,5e-6,
                    objF,
                    SolverSteep,
                    Derivator1(1e-5,"double"),
                    StepNorm("unit"),
                    SearchExtendedSteps(100),
                    UnstuckDont;
                    ϵgrad=0.,maxiter=Int(1e2),showtrace=true,
                    resettimer=true);

In [ ]:
idx = findfirst(x->x.objvalue==0.,hist)-1

x_ = cat((x->x.pos).(reverse(hist[1:idx-1]))...; dims=2)

In [ ]:
plot!(scan,(x_[1,:].-0.02125960840209658)/1e-3,(x_[2,:].-0.04351921680419316)/1e-3; c=:black)

In [ ]:
analyse(hist,trace,freqs)